## Build Keras Model and train mnist dataset from scratch

In [38]:
import tensorflow as tf
import numpy as np

In [42]:
## I have learn that the dense layers implement the following transformation on inputs:
    ## output = activation(dot(input , w)+b)
## where w and b are the model parameters
## w is the kernel attribute the bias attribute

In [16]:
## let's implement a layer object (NaiveDense) witch implement the function above
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval =0, maxval = 1e-1)
        self.w = tf.Variable(w_initial_value)
        
        b_shape = (output_size)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)
    
    def __call__(self, inputs):
            return self.activation(tf.matmul(inputs, self.w) + self.b)
        
    @property
    def weights(self):
        return [self.w , self.b]

In [13]:
#Now let's implement a NaiveSequential object wich embed our NaiveDense Models and 
# returns their weghts
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
    
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    
    @property
    def weights(self):
        weights = []
        for layer in  self.layers:
            weights += layer.weights
        return weights

In [17]:
## Now let's build our model
model = NaiveSequential([
    NaiveDense(input_size = 28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size = 512,     output_size=10, activation=tf.nn.softmax)
])

2022-09-28 16:53:22.638843: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-28 16:53:22.638959: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (s8d00-Lab): /proc/driver/nvidia/version does not exist


In [19]:
## assert that all two layers return two weights variable(w,b)
assert len(model.weights) == 4

In [32]:
#let's now focus on our data 
#precisely how we are going to split in into batches for our model
import math
class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
        
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images , labels
    

In [22]:
## Now let's implement a simple func to update batches and compute the loss
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [23]:
## Here 's the optimizer now
## You can also implement by 'hand' an alternative to it as:
###learning_rate = 1e-3
###def update_weights(gradients, weights):
###    for g, w in zip(gradients, weights):
###        w.assign_sub(g * learning_rate)
##I will use a keras optimizer here to "optimize" our results XD

from tensorflow.keras import optimizers
optimizer = optimizers.SGD(learning_rate=1e-3)
def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))


In [30]:
## Now let's implement the fit func for our model
##here we will update and fit batches , get the loss and update the model weights
def fit(model , images , labels , epochs , batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter} : {loss:.2f}")

In [28]:
## Load and rescale the data 
from tensorflow.keras.datasets import mnist
(train_images , train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images  = test_images.reshape((10000, 28 * 28))
test_images  = test_images.astype("float32") / 255

In [43]:
## start the training
history = fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0 : 0.63
loss at batch 100 : 0.67
loss at batch 200 : 0.57
loss at batch 300 : 0.63
loss at batch 400 : 0.70
Epoch 1
loss at batch 0 : 0.60
loss at batch 100 : 0.63
loss at batch 200 : 0.54
loss at batch 300 : 0.60
loss at batch 400 : 0.67
Epoch 2
loss at batch 0 : 0.57
loss at batch 100 : 0.60
loss at batch 200 : 0.51
loss at batch 300 : 0.57
loss at batch 400 : 0.65
Epoch 3
loss at batch 0 : 0.55
loss at batch 100 : 0.57
loss at batch 200 : 0.49
loss at batch 300 : 0.55
loss at batch 400 : 0.63
Epoch 4
loss at batch 0 : 0.52
loss at batch 100 : 0.55
loss at batch 200 : 0.47
loss at batch 300 : 0.53
loss at batch 400 : 0.61
Epoch 5
loss at batch 0 : 0.51
loss at batch 100 : 0.53
loss at batch 200 : 0.45
loss at batch 300 : 0.51
loss at batch 400 : 0.60
Epoch 6
loss at batch 0 : 0.49
loss at batch 100 : 0.51
loss at batch 200 : 0.43
loss at batch 300 : 0.50
loss at batch 400 : 0.59
Epoch 7
loss at batch 0 : 0.48
loss at batch 100 : 0.49
loss at batch 200 : 0.42
lo

In [34]:
#let's predict the test set
predictions = model(test_images)

In [41]:
## Now let's see how accurate our model was
#predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"Accuracy: {matches.mean():.2f}")

Accuracy: 0.81
